In [1]:
# Importing all the libraries
import pandas as pd
import string
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset, DataLoader, Dataset
import numpy as np
from sklearn import metrics
from torch.utils.tensorboard import SummaryWriter
from torchvision import transforms


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!unzip 'drive/MyDrive/Task2-dataset.zip' 

Archive:  drive/MyDrive/Task2-dataset.zip
   creating: Task2-dataset/
  inflating: Task2-dataset/x_test.npy  
  inflating: Task2-dataset/x_train.npy  
  inflating: Task2-dataset/y_test_age.npy  
  inflating: Task2-dataset/y_test_gender.npy  
  inflating: Task2-dataset/y_train_age.npy  
  inflating: Task2-dataset/y_train_gender.npy  


In [4]:
prefix = "Task2-dataset/"
x_train = np.load(prefix+"x_train.npy")
y_train_age = np.load(prefix+"y_train_age.npy")
y_train_gender = np.load(prefix+"y_train_gender.npy")

x_test = np.load(prefix+"x_test.npy")
y_test_age = np.load(prefix+"y_test_age.npy")
y_test_gender = np.load(prefix+"y_test_gender.npy")


In [5]:
y_train = np.zeros((y_train_gender.shape[0], 2))
for i in range(y_train_gender.shape[0]):
    y_train[i,1] = y_train_gender[i]
    y_train[i,0] = y_train_age[i]
     
y_test = np.zeros((y_test_gender.shape[0], 2))
for i in range(y_test_gender.shape[0]):
    y_test[i,1] = y_test_gender[i]
    y_test[i,0] = y_test_age[i]
     
print(y_train.shape)
print(y_test.shape)
# 1 age
# 2 gender

(6637, 2)
(6685, 2)


In [6]:
print(x_train.shape)

(6637, 128, 128, 3)


In [7]:
batch_size = 128
train_data = TensorDataset(torch.from_numpy(x_train), torch.from_numpy(y_train))
test_data = TensorDataset(torch.from_numpy(x_test), torch.from_numpy(y_test))

train_loader = DataLoader(train_data, shuffle=True, batch_size=batch_size, drop_last=True)
test_loader = DataLoader(test_data, shuffle=True, batch_size=batch_size, drop_last=True) 

In [8]:
# batch_size = 128

# x_train = x_train.reshape((-1, 1, 128, 128,3))
# x_train_tensor = torch.tensor(x_train)

# y_train = y_train.reshape(y_train.shape[0],2)
# y_train_tensor = torch.tensor(y_train)

# x_test_tensor = torch.tensor(x_test)
# y_test_tensor = torch.tensor(y_test)



# train_transform = transforms.Compose([
#                     ### Define augmentations 
#                     transforms.Resize((128,128)),
#                     transforms.ToTensor(),
#                     transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # https://discuss.pytorch.org/t/understanding-transform-normalize/21730
# ])

# test_transform = transforms.Compose([
#                     ### Define augmentations 
#                     transforms.Resize((128,128)),
#                     transforms.ToTensor(),
#                     transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # https://discuss.pytorch.org/t/understanding-transform-normalize/21730
# ])


# # Source: https://stackoverflow.com/questions/55588201/pytorch-transforms-on-tensordataset
# class CustomTensorDataset(Dataset):
#     """TensorDataset with support of transforms.
#     """
#     def __init__(self, tensors, transform=None):
#         # assert all(tensors[0].size(0) == tensor.size(0) for tensor in tensors)
#         self.tensors = tensors
#         self.transform = transform

#     def __getitem__(self, index):
#         x = self.tensors[0][index]

#         if self.transform:
#             x = self.transform(x)

#         y = self.tensors[1][index]

#         return x, y

#     def __len__(self):
#         return self.tensors[0].size(0)

# train_data_transform = CustomTensorDataset(tensors=(x_train_tensor, y_train_tensor), transform=train_transform)
# test_data_transform = CustomTensorDataset(tensors=(x_test_tensor, y_test_tensor), transform=test_transform)

# # train_data_transform = train_transform(train_data)
# # test_data_transform = train_transform(test_data)

# train_loader = DataLoader(train_data_transform, shuffle=True, batch_size=batch_size)
# test_loader = DataLoader(test_data_transform, shuffle=True, batch_size=batch_size) 

In [9]:
for i, data in enumerate(train_loader):
    # print(i, data[1].shape)
    print(i, data[0].shape, data[0].dtype)
    break

0 torch.Size([128, 128, 128, 3]) torch.uint8


In [27]:
# Source: https://discuss.pytorch.org/t/equivalent-of-keras-globalmaxpooling1d/45770/3
class GlobalMaxPooling1D(nn.Module):
    def __init__(self, data_format='channels_last'):
        super(GlobalMaxPooling1D, self).__init__()
        self.data_format = data_format
        self.step_axis = 1 if self.data_format == 'channels_last' else 2

    def forward(self, input):
        return torch.max(input, axis=self.step_axis).values


class BranchModel(torch.nn.Module):
    def __init__(self, embedding_dim, hidden_dim, n_layers=1, drop_prob=0.2):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.embedding_dim = embedding_dim
        self.n_layers = n_layers

        # self.embedding = nn.Embedding(vocab_size, embedding_dim)

        self.bi_lstm1 = nn.LSTM(embedding_dim, hidden_dim, num_layers=n_layers, dropout=drop_prob, batch_first=True, bidirectional=True)
        self.bi_lstm2 = nn.LSTM(embedding_dim, hidden_dim, num_layers=n_layers, dropout=drop_prob, batch_first=True, bidirectional=True)

        self.flip_horizontal = transforms.Compose([
                                                   transforms.RandomHorizontalFlip(p=1)
                                                 ])
        
        self.global_max_pooling = GlobalMaxPooling1D()

        self.fc1 = nn.Linear(1024,1)
        self.fc2 = nn.Linear(1024,1)

    def forward(self, x, hidden1, hidden2):
        inp1 = x
        inp2 = self.flip_horizontal(x)

        inp1_f = inp1.permute(0,4,1,2,3)
        inp1_f = torch.flatten(inp1_f, 2)
        lstm1_out, hidden1 = self.bi_lstm1(inp1_f.float(), hidden1)
        lstm1_out = self.global_max_pooling(lstm1_out)



        inp2_f = inp2.permute(0,4,1,2,3)
        inp2_f = torch.flatten(inp2_f, 2)
        lstm2_out, hidden2 = self.bi_lstm1(inp2_f.float(), hidden2)
        lstm2_out = self.global_max_pooling(lstm2_out)

        concat = torch.cat([lstm1_out, lstm2_out], 1)

        out1 = self.fc1(concat)
        out2 = self.fc2(concat)

        return out1, out2, hidden1, hidden2

    def init_hidden(self, batch_size):
        # print(self.n_layers, batch_size, self.hidden_dim)
        # return torch.zeros((self.n_layers, batch_size, self.hidden_dim)).to(device)
        h0 = torch.zeros((self.n_layers*2, batch_size, self.hidden_dim)).to(device)
        c0 = torch.zeros((self.n_layers*2, batch_size, self.hidden_dim)).to(device)
        hidden = (h0,c0)
        return hidden

EMBEDDING_DIM = 16384#128*128*3
HIDDEN_DIM = 256
N_LAYERS = 1
clip = 5

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = BranchModel(EMBEDDING_DIM, HIDDEN_DIM, N_LAYERS).to(device)
# loss_function1 = nn.L1Loss()
# loss_function2 = nn.BCELoss()

loss_function = nn.L1Loss()

lr = 0.002

optimizer = torch.optim.Adam(model.parameters(), lr=lr)
scheduler = None

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/rnn.py:65: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


In [28]:
def calculate_accuracy(y_pred, y):
    top_pred = y_pred.argmax(1, keepdim = True)
    correct = top_pred.eq(y.view_as(top_pred)).sum()
    acc = correct.float() / y.shape[0]
    return acc

def train(model,train_loader,loss_fn,optimizer,scheduler,device):
    accuracy = 0
    running_loss = 0
    total = 0
    correct = 0
    model.train()
    hidden1, hidden2 = model.init_hidden(batch_size), model.init_hidden(batch_size)

    for i, data in enumerate(train_loader):
        inputs, labels = data
        images,labels = inputs.to(device) , labels.to(device)
        images = images.unsqueeze(1)
 
        model.zero_grad()
        hidden1 = tuple([each.data for each in hidden1])
        hidden2 = tuple([each.data for each in hidden2])
        out1, out2, hidden1, hidde2 = model(images, hidden1, hidden2)
        # print(type(hidden1))
        outputs = torch.cat([out1, out2], 1)
        loss = loss_fn(outputs, labels)
        # print(labels[:,1].shape)
        # loss = loss_fn[0](out1.squeeze().float(), labels[:,0].float()) + loss_fn[1](out2.squeeze().float(), labels[:,1].float())*10

        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()* images.size(0)
        total += images.size(0)
        preds = torch.argmax(out2.data, 1)
        correct += (preds == labels[:,1]).sum().item()

        if i % 300 == 0:   
            print('[%d, %5d] Acc: %.3f loss: %.3f' %
                  (epoch + 1, i + 1, correct/total, running_loss /total))
            running_loss = 0.0
        # scheduler.step(loss/total)

def evaluate(model, loader, loss_fn, device):
    epoch_loss = 0
    epoch_acc = 0
    total = 0
    correct = 0
    model.eval()
    hidden1, hidden2 = model.init_hidden(batch_size), model.init_hidden(batch_size)
    with torch.no_grad():
        for i, data in enumerate(loader):
            inputs, labels = data
            inputs = inputs.unsqueeze(1)
            images,labels = inputs.to(device) , labels.to(device) 
            hidden1 = tuple([each.data for each in hidden1])
            hidden2 = tuple([each.data for each in hidden2])
            out1, out2, hidden1, hidde2 = model(images, hidden1, hidden2)
            # print(type(hidden1))
            outputs = torch.cat([out1, out2], 1)
            loss = loss_fn(outputs, labels)

            total += inputs.size(0)
            preds = torch.argmax(out2.data, 1)
            correct += (preds == labels[:,1]).sum().item()
            epoch_acc += correct/total
            epoch_loss += loss.item()

    print("Epoch %d Test Accuracy %.3f" % (epoch, epoch_acc/ len(loader)))

In [29]:
NUM_EPOCHS = 10
for epoch in range(NUM_EPOCHS):
    train(model, train_loader, loss_function, optimizer, scheduler, device)
    # train(model, train_loader, [loss_function1,loss_function2], optimizer, scheduler, device)
    evaluate(model, test_loader, loss_function, device)
    print("------------------------------------------------------------------------------")

[1,     1] Acc: 0.539 loss: 18.974
Epoch 0 Test Accuracy 0.560
------------------------------------------------------------------------------
[2,     1] Acc: 0.516 loss: 8.113
Epoch 1 Test Accuracy 0.557
------------------------------------------------------------------------------
[3,     1] Acc: 0.570 loss: 6.711
Epoch 2 Test Accuracy 0.558
------------------------------------------------------------------------------
[4,     1] Acc: 0.570 loss: 6.834
Epoch 3 Test Accuracy 0.556
------------------------------------------------------------------------------
[5,     1] Acc: 0.617 loss: 6.643
Epoch 4 Test Accuracy 0.564
------------------------------------------------------------------------------
[6,     1] Acc: 0.484 loss: 6.799
Epoch 5 Test Accuracy 0.556
------------------------------------------------------------------------------
[7,     1] Acc: 0.617 loss: 6.414
Epoch 6 Test Accuracy 0.556
------------------------------------------------------------------------------
[8,     1] A